In [2]:
import pytesseract
import cv2
import pyocr 
from PIL import Image
from PIL.ImageOps import invert
import matplotlib.pyplot as plt
import time
import numpy as np
import pytesseract

import imutils
from imutils.perspective import four_point_transform

ImportError: C extension: No module named 'pandas._libs.tslibs.conversion' not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace --force' to build the C extensions first.

In [178]:
img = cv2.imread("license/AA.jpg", cv2.IMREAD_COLOR)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 1440*900
bili = 800/img_gray.shape[0]
re_img_gray = cv2.resize(img_gray, dsize=(0,0), fx=bili, fy = bili)

_, img_gray_th = cv2.threshold(re_img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
poin_img = cv2.imread("license/A_point.jpg", cv2.IMREAD_GRAYSCALE)
_, poin_img_th = cv2.threshold(poin_img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

In [175]:
w, h = poin_img.shape[::-1]
res = cv2.matchTemplate(img_gray_th, poin_img_th, cv2.TM_CCORR_NORMED) 
threshold = res.max()
loc = np.where(res >= threshold)
for x,y in zip(*loc[::-1]):
    cv2.rectangle(re_img_gray, (x,y), (x+w, y+h), (0,0,255), 1)

In [177]:
cv2.imshow("poin_img_th",re_img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 对于证书的裁剪

In [2]:
image = cv2.imread('license/AA.jpg')
width1, height1 = image.shape[1],image.shape[0]

# bili = 800/image.shape[0]
# re_img_gray = cv2.resize(image, dsize=(0,0), fx=bili, fy = bili)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#高斯滤波
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#自适应二值化方法
blurred=cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,51,2)
'''
adaptiveThreshold函数：第一个参数src指原图像，原图像应该是灰度图。
    第二个参数x指当像素值高于（有时是小于）阈值时应该被赋予的新的像素值
    第三个参数adaptive_method 指： CV_ADAPTIVE_THRESH_MEAN_C 或 CV_ADAPTIVE_THRESH_GAUSSIAN_C
    第四个参数threshold_type  指取阈值类型：必须是下者之一 CV_THRESH_BINARY,• CV_THRESH_BINARY_INV
    第五个参数 block_size 指用来计算阈值的象素邻域大小: 3, 5, 7, ...
    第六个参数param1指与方法有关的参数。对方法CV_ADAPTIVE_THRESH_MEAN_C 和 CV_ADAPTIVE_THRESH_GAUSSIAN_C， 
    它是一个从均值或加权均值提取的常数, 尽管它可以是负数。
'''
#这一步可有可无，主要是增加一圈白框，以免刚好卷子边框压线后期边缘检测无果。好的样本图就不用考虑这种问题
blurred=cv2.copyMakeBorder(blurred,5,5,5,5,cv2.BORDER_CONSTANT,value=(255,255,255))

edged = cv2.Canny(blurred, 10, 100)
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
docCnt = None

if len(cnts) > 0:
    # 将轮廓按大小降序排序
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # 对排序后的轮廓循环处理
    for c in cnts:
        # 获取近似的轮廓
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # 如果近似轮廓有四个顶点，那么就认为找到了答题卡
        if len(approx) == 4:
            docCnt = approx
            break
            
newimage = image.copy()
for i in docCnt:
    #circle函数为在图像上作图，新建了一个图像用来演示四角选取
    cv2.circle(newimage, (i[0][0],i[0][1]), 10, (255, 0, 0), -1)
    
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

# 对灰度图应用二值化算法
thresh=cv2.adaptiveThreshold(warped,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,53,2)
#重塑可能用到的图像
thresh = cv2.resize(thresh, (width1, height1), cv2.INTER_LANCZOS4)
paper = cv2.resize(paper, (width1, height1), cv2.INTER_LANCZOS4)
warped = cv2.resize(warped, (width1, height1), cv2.INTER_LANCZOS4)
#均值滤波
# ChQImg = cv2.blur(thresh, (23, 23))
#二进制二值化
# ChQImg = cv2.threshold(ChQImg, 100, 225, cv2.THRESH_BINARY)[1]

cv2.imshow("edged",edged)
cv2.imshow("gray",gray)
cv2.imshow("newimage",newimage)
cv2.imshow("thresh",thresh)
cv2.imshow("paper",paper)
cv2.imshow("warped",warped)
cv2.imwrite("license/AAxwarped.jpg",warped)
cv2.imwrite("license/AAxpaper.jpg",paper)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [97]:
re_img_gray.shape

(800, 555, 3)

In [20]:
def inder(image):
    image = cv2.imread(image)
    img_location = {"qrcode": [1180,1420,65,295],
                    "uuid": [530,580,295,845],
                    "name": [615,660,280,860],
                    "type": [660,713,280,860],
                    "address": [713,763,280,860],
                    "legal": [763,810,280,860],
                    "capital": [810,860,280,860],
                    "start": [860,909,280,860],
                    "limit": [909,955,280,860],
                    "scope": [955,1180,280,950]}
    resout_json = {}
    for resout in img_location:
        if resout =="qrcode":
            continue
        print(img_location[resout][0],img_location[resout][1], img_location[resout][2], img_location[resout][3])
        resout_json[resout] = pytesseract.image_to_string(image[img_location[resout][0]: img_location[resout][1], img_location[resout][2]: img_location[resout][3]], lang='chi_sim')
    return resout_json
        cv2.imwrite(f"license/out/{resout}.jpg",image[img_location[resout][0]: img_location[resout][1], img_location[resout][2]: img_location[resout][3]])

#     cv2.imshow("RQ",image[1180:1420,65:295])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
image = 'license/resout.jpg'
inder(image)

530 580 295 845
615 660 280 860
660 713 280 860
713 763 280 860
763 810 280 860
810 860 280 860
860 909 280 860
909 955 280 860
955 1180 280 950


{'uuid': '统 一 社 会 信 用 代 码 91510100MA6AQPBR8H',
 'name': '成 都 西 辰 易 通 科 技 有 限 公 司',
 'type': '有 限 责 任 公 司 ( 自 然 人 投 资 或 控 股 的 法 人 独 资 )',
 'address': '成 都 高 新 区 科 园 三 路 4 号 1 栋 12 层 2 号',
 'legal': '张 义',
 'capital': '( 人 民 币 ) 壹 佰 万 元',
 'start': '2019 年 1 月 23 日',
 'limit': '2019 年 1 月 23 日 至 永 久',
 'scope': '计 算 机 软 件 技 术 开 发 、 技 术 咨 询 、 技 术 转 让 、 技 术 服 务 ; 电 子 产\n品 的 设 计 、 开 发 及 技 术 服 务 ; 批 发 计 算 机 及 零 配 件 、 电 子 产 品 、\n\n通 信 设 备 ( 不 含 无 线 广 播 电 视 发 射 及 卫 星 地 面 接 收 设 备 》; 信 息\n霞霾善瞳寰簧. ( 依 法 须 经 批 准 的 项 目 , 经 相 关 部 门 批 准 后 方 可 开'}

In [1]:
# -*- coding: utf-8 -*-
import pyzbar
from pyzbar.pyzbar import decode
from pyzbar.pyzbar import ZBarSymbol
import encodings

image_jpg = cv2.imread('license/resout.jpg')
re_image = cv2.resize(image_jpg,(1000, 1500))
image_png = cv2.cvtColor(re_image, cv2.COLOR_BGR2RGBA)
code2D = image_png[1180:1420,65:295]

# cv2.imwrite("code2221.png", code2D)

result = decode(code2D, symbols=[ZBarSymbol.QRCODE])
for info in result:
    print(info.type, "\n**\n",info.data.decode("utf-8"))

NameError: name 'cv2' is not defined

In [ ]:
result

In [4]:
cv2.imshow("code2D",code2D)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [58]:
ss = "喜欢"
ss.encode("utf-8").decode("utf-8")

'喜欢'

# 对于身份证的剪切

In [15]:
image1 = cv2.imread('people.jpg')
image = cv2.pyrMeanShiftFiltering(image1, 10, 15)
width1, height1 = image.shape[1],image.shape[0]

# bili = 800/image.shape[0]
# re_img_gray = cv2.resize(image, dsize=(0,0), fx=bili, fy = bili)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#高斯滤波
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#自适应二值化方法
blurred=cv2.adaptiveThreshold(blurred,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,51,2)
'''
adaptiveThreshold函数：第一个参数src指原图像，原图像应该是灰度图。
    第二个参数x指当像素值高于（有时是小于）阈值时应该被赋予的新的像素值
    第三个参数adaptive_method 指： CV_ADAPTIVE_THRESH_MEAN_C 或 CV_ADAPTIVE_THRESH_GAUSSIAN_C
    第四个参数threshold_type  指取阈值类型：必须是下者之一 CV_THRESH_BINARY,• CV_THRESH_BINARY_INV
    第五个参数 block_size 指用来计算阈值的象素邻域大小: 3, 5, 7, ...
    第六个参数param1指与方法有关的参数。对方法CV_ADAPTIVE_THRESH_MEAN_C 和 CV_ADAPTIVE_THRESH_GAUSSIAN_C， 
    它是一个从均值或加权均值提取的常数, 尽管它可以是负数。
'''
#这一步可有可无，主要是增加一圈白框，以免刚好卷子边框压线后期边缘检测无果。好的样本图就不用考虑这种问题
blurred=cv2.copyMakeBorder(blurred,5,5,5,5,cv2.BORDER_CONSTANT,value=(255,255,255))

edged = cv2.Canny(blurred, 10, 100)
cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
docCnt = None

if len(cnts) > 0:
    # 将轮廓按大小降序排序
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # 对排序后的轮廓循环处理
    for c in cnts:
        # 获取近似的轮廓
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # 如果近似轮廓有四个顶点，那么就认为找到了答题卡
        if len(approx) == 4:
            docCnt = approx
            break
            
newimage = image.copy()
for i in docCnt:
    #circle函数为在图像上作图，新建了一个图像用来演示四角选取
    cv2.circle(newimage, (i[0][0],i[0][1]), 10, (255, 0, 0), -1)
    
paper = four_point_transform(image1, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

# 对灰度图应用二值化算法
thresh=cv2.adaptiveThreshold(warped,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,53,2)
#重塑可能用到的图像
thresh = cv2.resize(thresh, (width1, height1), cv2.INTER_LANCZOS4)
paper = cv2.resize(paper, (width1, height1), cv2.INTER_LANCZOS4)
warped = cv2.resize(warped, (width1, height1), cv2.INTER_LANCZOS4)
#均值滤波
# ChQImg = cv2.blur(thresh, (23, 23))
#二进制二值化
# ChQImg = cv2.threshold(ChQImg, 100, 225, cv2.THRESH_BINARY)[1]

cv2.imshow("edged",edged)
cv2.imshow("gray",gray)
cv2.imshow("newimage",newimage)
cv2.imshow("thresh",thresh)
cv2.imshow("paper",paper)
cv2.imshow("warped",warped)
cv2.imwrite("AAxwarped.jpg",warped)
cv2.imwrite("AAxpaper.jpg",paper)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)
while(1):
    _, frame = cap.read()
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     lower_red = np.array([30,150,50])
#     upper_red = np.array([255,255,180])
#     mask = cv2.inRange(hsv, lower_red, upper_red)
#     res = cv2.bitwise_and(frame,frame, mask= mask)
    cv2.imshow('Original',frame)
#     edges = cv2.Canny(frame,100,200)
#     cv2.imshow('Edges',edges)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
cv2.destroyAllWindows()
cap.release()